# Vorarbeiten

In [3]:
import pandas as pd
import numpy as np

## Umgebungsvariablen

In [4]:
failure_datapath = '''D:/jupyter-notebooks/LANL_Earthquake_Prediction/failure/'''
train_data_path = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/all/train.csv'''

## Trainingsdaten laden

In [5]:
train_data = pd.read_csv(train_data_path, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

# Feature Extraction

Die Testsegmente bestehen jeweils aus 150.000 Messpunkte. Aus diesem Grund setzten wir die Fenstergrösse auch auf 150000 Messpunkte.

In [102]:
def generate_featureRow(acoustic_data, time_to_failure):
    
    strain_feature = []
    time_to_failure = []
    strain_feature.append(acoustic_data.mean())
    strain_feature.append(acoustic_data.std())
    strain_feature.append(acoustic_data.kurtosis())
    strain_feature.append(acoustic_data.skew())
    strain_feature.append(acoustic_data.min())
    strain_feature.append(acoustic_data.max())
    strain_feature.append(acoustic_data.quantile(0.01, interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.05,interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.95,interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.99,interpolation='midpoint'))
    strain_feature.append(acoustic_data.abs().max())
    strain_feature.append(acoustic_data.abs().mean())
    strain_feature.append(acoustic_data.abs().std())
    
    time_to_failure.append(time_to_failure.values[-1])
    return strain_feature, time_to_failure

Das Fenster, mit der grösse 150000), wird durch das Trainingssignal geschoben mit einer geplanten Überlappung von 80%

In [103]:
def generate_Features(train_data,step, windows_size=150000, ):
    init_idx = 0
    features = np.array([], dtype=np.float64)
    time_to_failure = np.array([], dtype=np.float64)

    while (init_idx + window_size < len(train)):
        window = train.iloc[init_idx:init_idx + window_size]
        x_train , y_train =  generate_featureRow(window.acoustic_data,window.time_to_failure)
        features.append(x_train)
        time_to_failure.append(y_train)
        init_idx += step

    return featuers, time_to_failure

In [101]:
'''
def generate_Features(train_data,step, windows_size=150000, ):
    init_idx = 0
    features = []

    while (init_idx + window_size < len(train)):
        window = train.iloc[init_idx:init_idx + window_size]
        features.append(generate_featureRow(window.acoustic_data,window.time_to_failure))
        init_idx += step

    return pd.DataFrame(features)
'''

'\ndef generate_Features(train_data,step, windows_size=150000, ):\n    init_idx = 0\n    features = []\n\n    while (init_idx + window_size < len(train)):\n        window = train.iloc[init_idx:init_idx + window_size]\n        features.append(generate_featureRow(window.acoustic_data,window.time_to_failure))\n        init_idx += step\n\n    return pd.DataFrame(features)\n'

In [107]:
step = int(0.9*150000)
features, time_to_failure  = generate_features(train_data,step)

AttributeError: 'int' object has no attribute 'values'

In [108]:
time_to_failure.shape

NameError: name 'time_to_failure' is not defined

# Create Model Workbench

## Imports

In [123]:
# Scaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Model selection
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Modell
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

## RandomForrest and GradientBoosting

In [118]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', GradientBoostingRegressor())])

In [119]:
param_grid = [
    {'classifier': [GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
      max_depth=1, random_state=0, loss='ls')], 'preprocessing': [None],
     'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},
    {'classifier': [RandomForestRegressor(n_estimators=100,min_samples_split=30,max_features=13)],
     'preprocessing': [None]}]

In [121]:
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, random_state=0)

grid = GridSearchCV(pipe, param_grid, scoring='mean_absolute_error', cv=5)
grid.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

NameError: name 'train_test_split' is not defined